In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import style
import seaborn as sns

matplotlib.rc('font', family='NanumGothic')
matplotlib.rcParams['axes.unicode_minus'] = False

In [2]:
df_product = pd.read_csv("./DataSet/master_product_sheet_001.csv", engine='python')
df_product.head(2)

,Unnamed: 0,#,공급사,과세/면세,담기,매입가,상품명(기본),상품코드,재입고알림,카테고리,판매가,품절여부,후기
0,0,1,패커스,과세,0,55440,"실링용기 71호 백색 2,200ml 400개 1박스",2184,사용함,018 : 실링용기\n018002 : 실링용기 > 2319(2318)시리즈\n042...,77480,정상,0
1,1,2,(주)니드코,과세,0,3080,"니드코]국산제품 정식쟁반,병원식판,분식점 돈까스쟁반 업소용 화이트 항균아트쟁반 5호",2181,사용안함,002 : 주방용품\n002002 : 주방용품 > 주방용품\n046 : 주방용품/소...,4950,정상,0


In [3]:
df_product.shape

(1017, 13)

In [4]:
def divide_category(x):
    if '용기' in x['카테고리']:
        return '용기' # 1. 용기(용기 + 실링용기)
    
    elif '식자재' in x['카테고리']:
        return '식자재' # 2. 식자재
    
    elif '실링필름' in x['카테고리']:
        return '실링제품' # 4. 실링제품 (실링필름 + 실링기계)
    
    elif '실링기계' in x['카테고리']:
        return '실링제품'
    
    elif '주방용품' in x['카테고리']:
        return '기타'  # 6. 기타(주방용품, 빨대, 스티커)
    
    elif '빨대' in x['카테고리']:
        return '기타'
    
    elif '스티커' in x['카테고리']:
        return '기타'
    
    elif '카달로그' in x['카테고리']:
        if '용기' in x['상품명(기본)']:
            return '용기'
        
        return '기타'  
    
    elif '소량' in x['카테고리']:
        return '용기'
    
    else:
        return -1
# 주방용품 안에 일회용품 포함
# 카테고리 소량 => 실링 PP 제품 => 실링 용기라고 판단 => 용기에 포함
# 카테고리 카달로그 => 용기는 용기, 냅킨 80밴드는 구매이력 order_list에 없음(100밴드 등은 존재하므로 이건 삭제). product 목록에서 제외

In [5]:
df_product['분류카테고리'] = df_product.T.apply(lambda x: divide_category(x))
df_product.head(2)

,Unnamed: 0,#,공급사,과세/면세,담기,매입가,상품명(기본),상품코드,재입고알림,카테고리,판매가,품절여부,후기,분류카테고리
0,0,1,패커스,과세,0,55440,"실링용기 71호 백색 2,200ml 400개 1박스",2184,사용함,018 : 실링용기\n018002 : 실링용기 > 2319(2318)시리즈\n042...,77480,정상,0,용기
1,1,2,(주)니드코,과세,0,3080,"니드코]국산제품 정식쟁반,병원식판,분식점 돈까스쟁반 업소용 화이트 항균아트쟁반 5호",2181,사용안함,002 : 주방용품\n002002 : 주방용품 > 주방용품\n046 : 주방용품/소...,4950,정상,0,기타


In [6]:
# 삭제할 row index 찾기
# 미소크린 테이블냅킨 74매*80밴드
df_product.loc[df_product['상품명(기본)'] == '미소크린 테이블냅킨 74매*80밴드']

,Unnamed: 0,#,공급사,과세/면세,담기,매입가,상품명(기본),상품코드,재입고알림,카테고리,판매가,품절여부,후기,분류카테고리
189,190,191,패커스,과세,0,8480,미소크린 테이블냅킨 74매*80밴드,1968,사용안함,061 : vol.4 Catalogue\n061012 : vol.4 Catalogu...,9500,정상,0,기타


In [7]:
# row 삭제
# 미소크린 테이블냅킨 74매*80밴드 
df_product.drop([189], inplace=True)
df_product.shape

(1016, 14)

In [8]:
df_check = (df_product['분류카테고리'] == -1)

In [9]:
df_check.value_counts() # 모두 분류 잘 완료 됨을 확인

False    1016
Name: 분류카테고리, dtype: int64

## log_order와 master_product 합치기

In [10]:
df_order = pd.read_csv("./DataSet/log_order_sheet_001.csv", engine='python')
df_order.head(2)

,Unnamed: 0,결제수단,공급원가,교환구분,네이버 포인트,배송번호,배송비 정보,배송시작일,배송완료일,사용한 적립금액,...,주문자ID,카드사,쿠폰 할인금액,판매가,품목번호,친환경,사은품,회원,주문수량,총수량
0,0,무통장입금,44550.0,교환안함,0,D-20181227-0000648-00,무료,2019-01-07 오전 8:56,2019-01-11 오전 5:32,0,...,pp683,기타,0,71450,42602,False,False,True,1,300
1,1,무통장입금,44000.0,교환안함,0,D-20181229-0000119-00,무료,2019-01-03 오전 9:30,2019-01-07 오전 5:32,0,...,asd0908,기타,0,70620,42930,False,False,True,2,2000


In [11]:
df_order.columns

Index(['Unnamed: 0', '결제수단', '공급원가', '교환구분', '네이버 포인트', '배송번호', '배송비 정보',
       '배송시작일', '배송완료일', '사용한 적립금액', '상품구매금액', '상품명', '상품번호', '상품별 추가할인금액',
       '수령인 주소', '주문경로', '주문번호', '주문일시', '주문자ID', '카드사', '쿠폰 할인금액', '판매가',
       '품목번호', '친환경', '사은품', '회원', '주문수량', '총수량'],
      dtype='object')

In [12]:
df_product_category = df_product[['상품명(기본)', '분류카테고리']]
df_product_category.rename(columns={'상품명(기본)': '상품명'}, inplace=True)
df_product_category

/home/piai/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,상품명,분류카테고리
0,"실링용기 71호 백색 2,200ml 400개 1박스",용기
1,"니드코]국산제품 정식쟁반,병원식판,분식점 돈까스쟁반 업소용 화이트 항균아트쟁반 5호",기타
2,"니드코]QueenRose 장미 밥공기그릇 /중국집,주방,식당,가정용 깨지지 않는 단...",기타
3,니드코 터치미 종이컵디스펜서(자석형) 종이컵보관함,기타
4,니드코 뉴카멜리아 사각수저함(수저통),기타
...,...,...
1012,"95파이 다용도컵 세트 4호 소 200ml 백색 PS 1,000개 1set(2박스)",용기
1013,"75파이 다용도컵 세트 3호 대 165ml 백색 PS 3,000개 1set(2박스)",용기
1014,"75파이 다용도컵 세트 3호 소 150ml 백색 PS 3,000개 1set(2박스)",용기
1015,"70파이 다용도컵 세트 2호 대 80ml 백색 PS 3,000개 1set(2박스)",용기


In [14]:
# type(df_product_category['상품번호'][1])

In [ ]:
# type(df_order['상품번호'][1])

In [15]:
type(df_product_category['상품명'][1])

str

In [16]:
type(df_order['상품명'][1])

str

In [ ]:
# 상품번호 str => float 형태로 바꿔주기

# df_order[df_order["상품번호"] == "2017년 상품번호"]["상품번호"] = -1
df_order.loc[df_order["상품번호"] == "2017년 상품번호", "상품번호"] = -1

df_order[df_order["상품번호"] == -1]["상품명"]

In [17]:
type(df_order['상품번호'][1])

str

In [18]:
type(df_product['상품코드'][1])

numpy.int64

In [39]:
df_order['상품번호']

0        1077.0
1          42.0
2        1271.0
3        1085.0
4        1106.0
          ...  
46744     445.0
46745     172.0
46746     140.0
46747     184.0
46748     479.0
Name: 상품번호, Length: 46749, dtype: object

In [58]:
df_order = df_order['상품번호'].apply(lambda x: str(x).rstrip('.0'))
df_order['상품번호']

1077.0
42.0
1271.0
1085.0
1106.0
1374.0
1071.0
1373.0
1374.0
1037.0
1058.0
1059.0
1071.0
1373.0
1374.0
1037.0
1373.0
1374.0
1037.0
1374.0
1077.0
68.0
603.0
608.0
455.0
1322.0
1275.0
1373.0
1374.0
1205.0
1374.0
1373.0
455.0
455.0
1106.0
1373.0
1374.0
1070.0
1230.0
1077.0
92.0
1283.0
961.0
949.0
110.0
382.0
381.0
38.0
1071.0
1106.0
1283.0
1037.0
1180.0
1374.0
605.0
1324.0
603.0
603.0
1350.0
1233.0
1235.0
378.0
447.0
602.0
608.0
1271.0
1077.0
455.0
603.0
1077.0
1077.0
1373.0
1374.0
78.0
1059.0
1106.0
1373.0
1374.0
1059.0
1055.0
1373.0
1233.0
1055.0
1374.0
1373.0
42.0
248.0
455.0
382.0
381.0
1270.0
90.0
1205.0
1105.0
1037.0
1374.0
1373.0
603.0
455.0
1374.0
378.0
1334.0
1282.0
1059.0
1106.0
1374.0
70.0
1231.0
1235.0
94.0
1055.0
1373.0
1374.0
592.0
593.0
38.0
592.0
593.0
446.0
1373.0
1374.0
1374.0
1374.0
1373.0
964.0
1077.0
1275.0
455.0
1077.0
608.0
437.0
1167.0
1217.0
1270.0
1217.0
1232.0
1060.0
602.0
445.0
92.0
605.0
603.0
88.0
94.0
608.0
1077.0
217.0
1231.0
1071.0
962.0
949.0
1205.0
1204.

962.0
248.0
966.0
1318.0
1070.0
94.0
1168.0
455.0
172.0
1373.0
1077.0
1374.0
1373.0
603.0
455.0
1375.0
1374.0
1373.0
90.0
45.0
1272.0
1342.0
639.0
207.0
397.0
217.0
392.0
46.0
1106.0
1375.0
1374.0
455.0
1071.0
1058.0
1373.0
1059.0
1374.0
455.0
1106.0
1070.0
1071.0
1091.0
1375.0
1374.0
1373.0
592.0
593.0
1105.0
1374.0
252.0
559.0
70.0
936.0
608.0
94.0
68.0
1374.0
1312.0
1327.0
1234.0
1317.0
1106.0
1373.0
1082.0
608.0
188.0
479.0
455.0
184.0
1169.0
437.0
603.0
957.0
962.0
966.0
104.0
44.0
1077.0
1334.0
104.0
1084.0
603.0
1373.0
1374.0
140.0
1373.0
1374.0
1295.0
1375.0
1374.0
1373.0
90.0
1077.0
1350.0
42.0
43.0
1374.0
1139.0
172.0
1106.0
1373.0
1374.0
1373.0
1374.0
608.0
1374.0
1373.0
953.0
258.0
1373.0
1060.0
451.0
1106.0
455.0
602.0
1139.0
43.0
90.0
78.0
1265.0
455.0
1069.0
44.0
42.0
1180.0
1071.0
1214.0
603.0
455.0
1072.0
1069.0
1373.0
1374.0
1077.0
1167.0
966.0
1071.0
1085.0
608.0
450.0
94.0
1283.0
603.0
1083.0
437.0
1106.0
1374.0
1373.0
455.0
597.0
1127.0
190.0
1040.0
1231.0
603.0
11

184.0
1091.0
1070.0
603.0
1282.0
1536.0
1232.0
1106.0
1374.0
1373.0
172.0
447.0
481.0
172.0
82.0
378.0
43.0
90.0
933.0
932.0
42.0
602.0
94.0
1325.0
250.0
1315.0
608.0
380.0
1275.0
1282.0
603.0
1283.0
1091.0
1077.0
455.0
478.0
1374.0
1373.0
455.0
455.0
38.0
1055.0
1374.0
1373.0
1232.0
40.0
1077.0
603.0
1085.0
42.0
455.0
72.0
1055.0
1358.0
1350.0
1077.0
455.0
1277.0
481.0
1121.0
1329.0
1297.0
1350.0
1275.0
1282.0
603.0
90.0
382.0
603.0
1350.0
455.0
605.0
1374.0
1106.0
69.0
1374.0
1373.0
1059.0
1106.0
1059.0
1106.0
1373.0
1374.0
47.0
1375.0
1373.0
1374.0
1071.0
945.0
1374.0
1283.0
1234.0
1374.0
483.0
962.0
957.0
455.0
603.0
90.0
1312.0
1077.0
603.0
437.0
455.0
1346.0
1070.0
1038.0
1373.0
1374.0
1269.0
1341.0
957.0
1234.0
455.0
451.0
45.0
1283.0
1091.0
608.0
1077.0
378.0
1201.0
90.0
381.0
1106.0
1373.0
1297.0
1283.0
1203.0
447.0
1181.0
1283.0
1071.0
1282.0
45.0
1077.0
1226.0
381.0
90.0
1077.0
1282.0
1317.0
603.0
608.0
1091.0
1283.0
603.0
1342.0
480.0
381.0
1083.0
47.0
445.0
90.0
1127.0
603

455.0
1374.0
1059.0
1058.0
1106.0
1375.0
1374.0
1373.0
1055.0
1374.0
1373.0
42.0
1089.0
379.0
1205.0
1071.0
1539.0
608.0
1181.0
1059.0
1374.0
1373.0
1266.0
1373.0
1071.0
1374.0
1375.0
1070.0
1373.0
1122.0
1374.0
140.0
1297.0
1205.0
1267.0
455.0
966.0
449.0
437.0
1402.0
68.0
1315.0
608.0
1282.0
1283.0
1124.0
966.0
1316.0
957.0
248.0
72.0
447.0
1085.0
78.0
437.0
40.0
47.0
1227.0
602.0
602.0
608.0
1077.0
188.0
1085.0
1297.0
1091.0
602.0
1070.0
1038.0
608.0
1077.0
90.0
1374.0
1375.0
1373.0
1540.0
603.0
86.0
1077.0
1069.0
379.0
1374.0
1373.0
603.0
1321.0
1267.0
1268.0
1312.0
250.0
188.0
207.0
397.0
1302.0
1283.0
1055.0
1106.0
455.0
1501.0
1205.0
608.0
1373.0
455.0
1077.0
1059.0
381.0
928.0
1077.0
1077.0
172.0
608.0
603.0
1374.0
40.0
1316.0
1040.0
608.0
82.0
381.0
1373.0
190.0
1358.0
1359.0
441.0
1374.0
608.0
602.0
1069.0
602.0
603.0
437.0
1189.0
1267.0
608.0
1077.0
1106.0
1373.0
1374.0
446.0
96.0
88.0
248.0
812.0
962.0
1350.0
1077.0
1282.0
1267.0
608.0
1077.0
378.0
455.0
72.0
70.0
451.0
120

1522.0
605.0
446.0
478.0
437.0
455.0
68.0
455.0
1373.0
447.0
1127.0
608.0
1122.0
1070.0
1225.0
1069.0
1227.0
1069.0
603.0
928.0
937.0
479.0
1085.0
39.0
44.0
603.0
483.0
1326.0
38.0
437.0
602.0
378.0
1121.0
84.0
86.0
1373.0
295.0
455.0
1539.0
1077.0
1374.0
1373.0
1139.0
1374.0
1085.0
451.0
1071.0
254.0
1085.0
1274.0
1374.0
936.0
1274.0
455.0
603.0
1226.0
447.0
1071.0
962.0
1373.0
1374.0
449.0
1069.0
603.0
90.0
1374.0
1373.0
1139.0
1374.0
1373.0
47.0
603.0
1373.0
437.0
962.0
602.0
1106.0
1373.0
1374.0
45.0
1374.0
1373.0
1055.0
1325.0
1374.0
1373.0
1374.0
391.0
50.0
1038.0
1374.0
1373.0
378.0
380.0
379.0
378.0
1374.0
1373.0
1374.0
446.0
1374.0
1374.0
1375.0
451.0
94.0
1277.0
1235.0
455.0
1373.0
1077.0
45.0
1121.0
1278.0
1277.0
1374.0
1373.0
450.0
1297.0
1342.0
1072.0
603.0
70.0
1055.0
1374.0
1373.0
1374.0
1373.0
1202.0
68.0
1343.0
1071.0
1106.0
1059.0
1374.0
1373.0
1374.0
608.0
1316.0
78.0
44.0
92.0
603.0
437.0
96.0
90.0
1077.0
455.0
608.0
455.0
1517.0
160.0
455.0
90.0
90.0
104.0
962.0
12

...

1238.0
1209.0
1085.0
1237.0
1035.0
352.0
455.0
92.0
1238.0
1035.0
1036.0
1238.0
602.0
1237.0
1036.0
355.0
1084.0
382.0
355.0
1035.0
1036.0
1312.0
1209.0
1205.0
1237.0
41.0
1139.0
1036.0
1237.0
608.0
1205.0
584.0
1077.0
1237.0
1268.0
1205.0
1237.0
966.0
1209.0
47.0
1237.0
378.0
352.0
1106.0
1058.0
1238.0
45.0
42.0
1238.0
1265.0
90.0
931.0
1243.0
455.0
355.0
608.0
602.0
355.0
437.0
352.0
455.0
1237.0
1077.0
248.0
962.0
966.0
1243.0
1209.0
1071.0
1035.0
1036.0
1238.0
254.0
1234.0
352.0
68.0
584.0
1230.0
446.0
1237.0
602.0
90.0
1077.0
479.0
1106.0
1035.0
1209.0
355.0
1266.0
437.0
1209.0
445.0
1237.0
1238.0
445.0
1209.0
608.0
1077.0
584.0
1271.0
1180.0
1035.0
1237.0
608.0
1122.0
352.0
94.0
584.0
1266.0
1267.0
1269.0
1270.0
1271.0
1272.0
1268.0
1238.0
605.0
88.0
442.0
1209.0
1085.0
355.0
40.0
1209.0
443.0
1209.0
603.0
1070.0
455.0
1209.0
608.0
42.0
584.0
1169.0
1209.0
380.0
1083.0
1238.0
380.0
379.0
378.0
1037.0
355.0
380.0
352.0
480.0
1237.0
378.0
1069.0
355.0
378.0
379.0
380.0
355.0
608.0


88.0
38.0
1077.0
1237.0
447.0
603.0
1238.0
480.0
352.0
1270.0
1139.0
1037.0
966.0
1180.0
38.0
1237.0
1238.0
1077.0
1055.0
1238.0
605.0
355.0
1203.0
1204.0
76.0
1237.0
455.0
1237.0
602.0
1209.0
1077.0
352.0
1167.0
603.0
1238.0
172.0
1237.0
1036.0
608.0
352.0
109.0
94.0
1209.0
608.0
40.0
1214.0
447.0
1131.0
1266.0
380.0
1237.0
382.0
1237.0
379.0
1209.0
608.0
1209.0
94.0
355.0
1190.0
449.0
1202.0
1238.0
964.0
1233.0
584.0
1260.0
1297.0
1350.0
1270.0
1265.0
1071.0
1218.0
1214.0
1085.0
94.0
1238.0
1036.0
1035.0
1069.0
1238.0
1036.0
1209.0
380.0
352.0
1302.0
1357.0
1358.0
1359.0
1209.0
1059.0
1071.0
1036.0
1035.0
1238.0
90.0
1238.0
1282.0
1209.0
90.0
1238.0
45.0
1237.0
480.0
1237.0
1202.0
38.0
42.0
1077.0
1209.0
1282.0
608.0
355.0
1276.0
1243.0
603.0
352.0
142.0
378.0
1077.0
352.0
42.0
1237.0
186.0
397.0
352.0
352.0
378.0
1282.0
1106.0
1237.0
1071.0
352.0
1169.0
110.0
1209.0
483.0
1237.0
1085.0
188.0
1238.0
455.0
1237.0
1323.0
1232.0
1209.0
45.0
68.0
355.0
47.0
1209.0
608.0
584.0
1055.0
1283

1312.0
1209.0
1266.0
602.0
352.0
378.0
1209.0
947.0
1237.0
447.0
1237.0
478.0
1123.0
1209.0
1209.0
45.0
352.0
92.0
355.0
455.0
1237.0
42.0
602.0
1209.0
605.0
584.0
378.0
352.0
455.0
1373.0
1238.0
94.0
1341.0
1238.0
352.0
1205.0
1269.0
1266.0
1209.0
92.0
1238.0
603.0
447.0
1238.0
1238.0
1077.0
1209.0
1275.0
352.0
94.0
1209.0
1282.0
1237.0
1205.0
1237.0
1085.0
352.0
1084.0
1238.0
1373.0
1374.0
355.0
391.0
1237.0
1325.0
1237.0
1230.0
1077.0
1282.0
1238.0
1374.0
1373.0
1238.0
455.0
1312.0
1209.0
1374.0
1373.0
1237.0
479.0
1152.0
1237.0
523.0
962.0
190.0
352.0
602.0
1209.0
608.0
84.0
82.0
1203.0
1204.0
1243.0
1058.0
1059.0
1238.0
455.0
1238.0
1077.0
1297.0
1238.0
92.0
1238.0
379.0
378.0
602.0
447.0
88.0
355.0
455.0
1237.0
1059.0
1069.0
1374.0
1373.0
355.0
440.0
1209.0
478.0
352.0
44.0
1209.0
1077.0
1283.0
1282.0
1237.0
70.0
355.0
605.0
1238.0
1297.0
1205.0
88.0
1209.0
608.0
352.0
455.0
1237.0
90.0
1238.0
1105.0
1243.0
381.0
1209.0
90.0
1238.0
608.0
188.0
1209.0
1373.0
1237.0
252.0
352.0
481

2017년 상품번호
2017년 상품번호
215.0
45.0
2017년 상품번호
2017년 상품번호
90.0
393.0
386.0
397.0
39.0
391.0
397.0
172.0
88.0
90.0
88.0
45.0
62.0
379.0
378.0
57.0
184.0
174.0
38.0
42.0
45.0
217.0
43.0
395.0
391.0
174.0
42.0
188.0
64.0
89.0
53.0
188.0
215.0
391.0
207.0
90.0
172.0
392.0
172.0
207.0
394.0
92.0
207.0
397.0
205.0
395.0
205.0
395.0
168.0
216.0
403.0
388.0
174.0
217.0
388.0
215.0
391.0
217.0
391.0
61.0
62.0
393.0
392.0
215.0
379.0
172.0
84.0
170.0
46.0
375.0
174.0
62.0
170.0
187.0
168.0
160.0
43.0
38.0
44.0
64.0
207.0
396.0
158.0
48.0
395.0
174.0
95.0
91.0
93.0
395.0
205.0
395.0
248.0
174.0
41.0
43.0
184.0
188.0
47.0
45.0
94.0
40.0
66.0
47.0
42.0
58.0
205.0
395.0
393.0
205.0
396.0
205.0
219.0
388.0
45.0
216.0
404.0
207.0
397.0
38.0
170.0
207.0
394.0
140.0
53.0
55.0
89.0
174.0
164.0
174.0
179.0
205.0
395.0
172.0
215.0
391.0
187.0
405.0
246.0
217.0
393.0
44.0
187.0
39.0
44.0
40.0
219.0
388.0
207.0
396.0
405.0
207.0
395.0
170.0
88.0
90.0
40.0
188.0
184.0
88.0
377.0
207.0
397.0
208.0
411.0
42.0
172.

0        1077.0
1          42.0
2        1271.0
3        1085.0
4        1106.0
          ...  
46744     445.0
46745     172.0
46746     140.0
46747     184.0
46748     479.0
Name: 상품번호, Length: 46749, dtype: object

In [44]:
type(df_product['상품코드'][1])

numpy.int64

In [19]:
df_product['상품코드'] = df_product['상품코드'].astype(str)

In [20]:
type(df_product['상품코드'][1])

str

In [22]:
df_product['상품코드'] = df_product['상품코드'].T.apply(lambda x: x + ".0")
df_product['상품코드']

0       2184.0
1       2181.0
2       2180.0
3       2179.0
4       2178.0
         ...  
1012      42.0
1013      41.0
1014      40.0
1015      39.0
1016      38.0
Name: 상품코드, Length: 1016, dtype: object

In [23]:
df_order['상품번호']

0        1077.0
1          42.0
2        1271.0
3        1085.0
4        1106.0
          ...  
46744     445.0
46745     172.0
46746     140.0
46747     184.0
46748     479.0
Name: 상품번호, Length: 46749, dtype: object

In [24]:
# df_order['상품번호'] = df_order['상품번호'].astype(float).apply(np.int64)
# df_order['상품번호'] = df_order['상품번호'].astype(float)

In [25]:
# df_product_category['상품번호'] = df_product_category['상품번호'].astype(float).apply(np.int64)

In [ ]:
# 데이터 형태 맞춰주기
# df_order['상품번호'] = df_order['상품번호'].astype(float)
# 데이터 형태 맞춰주기
# df_product_category['상품번호'] = df_product_category['상품번호'].astype(float)

In [26]:
# type(df_product_category['상품번호'][1])

In [ ]:
type(df_order['상품번호'][0])

In [27]:
df_order

,Unnamed: 0,결제수단,공급원가,교환구분,네이버 포인트,배송번호,배송비 정보,배송시작일,배송완료일,사용한 적립금액,...,주문자ID,카드사,쿠폰 할인금액,판매가,품목번호,친환경,사은품,회원,주문수량,총수량
0,0,무통장입금,44550.0,교환안함,0,D-20181227-0000648-00,무료,2019-01-07 오전 8:56,2019-01-11 오전 5:32,0,...,pp683,기타,0,71450,42602,False,False,True,1,300
1,1,무통장입금,44000.0,교환안함,0,D-20181229-0000119-00,무료,2019-01-03 오전 9:30,2019-01-07 오전 5:32,0,...,asd0908,기타,0,70620,42930,False,False,True,2,2000
2,2,무통장입금,6820.0,교환안함,0,D-20181230-0000100-00,무료,2019-01-03 오전 9:30,2019-01-07 오전 5:32,0,...,NaN,기타,0,13910,42998,False,False,False,1,200
3,3,무통장입금,49500.0,교환안함,0,D-20181231-0000087-00,무료,2019-01-03 오전 9:30,2019-01-07 오전 5:32,0,...,23533593@n,기타,0,81600,43063,False,False,True,1,300
4,4,무통장입금,61600.0,교환안함,0,D-20181231-0000108-00,무료,2019-01-07 오전 8:56,2019-01-11 오전 5:32,0,...,romandoll,기타,0,94710,43066,False,False,True,1,400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46744,46744,카드,36300.0,교환안함,0,D-20171229-0000036-00,무료,2017-12-29 오후 4:32,2018-01-02 오전 5:31,0,...,73987748@n,국민,0,52800,3170,False,False,True,1,200
46745,46745,카드,36300.0,교환안함,0,D-20171229-0000043-00,무료,2018-01-03 오전 9:45,2018-01-07 오전 5:38,0,...,chuheesuk,비씨,0,56100,3172,False,False,True,2,400
46746,46746,카드,22280.0,교환안함,300,D-20171230-0000022-01,무료,2018-01-02 오후 7:06,2018-01-11 오전 2:46,0,...,NaN,기타,0,36000,3177,False,False,False,1,1500
46747,46747,카드,55000.0,교환안함,320,D-20171231-0000017-01,무료,2018-01-03 오후 2:00,2018-01-06 오후 6:06,0,...,NaN,기타,0,73460,3178,False,False,False,1,400


In [28]:
df_merge = pd.merge(df_order, df_product_category, on=['상품명'], how='left')type(df_product['상품코드'][1])

In [29]:
df_merge

,Unnamed: 0,결제수단,공급원가,교환구분,네이버 포인트,배송번호,배송비 정보,배송시작일,배송완료일,사용한 적립금액,...,카드사,쿠폰 할인금액,판매가,품목번호,친환경,사은품,회원,주문수량,총수량,분류카테고리
0,0,무통장입금,44550.0,교환안함,0,D-20181227-0000648-00,무료,2019-01-07 오전 8:56,2019-01-11 오전 5:32,0,...,기타,0,71450,42602,False,False,True,1,300,용기
1,1,무통장입금,44000.0,교환안함,0,D-20181229-0000119-00,무료,2019-01-03 오전 9:30,2019-01-07 오전 5:32,0,...,기타,0,70620,42930,False,False,True,2,2000,용기
2,2,무통장입금,6820.0,교환안함,0,D-20181230-0000100-00,무료,2019-01-03 오전 9:30,2019-01-07 오전 5:32,0,...,기타,0,13910,42998,False,False,False,1,200,기타
3,3,무통장입금,49500.0,교환안함,0,D-20181231-0000087-00,무료,2019-01-03 오전 9:30,2019-01-07 오전 5:32,0,...,기타,0,81600,43063,False,False,True,1,300,용기
4,4,무통장입금,61600.0,교환안함,0,D-20181231-0000108-00,무료,2019-01-07 오전 8:56,2019-01-11 오전 5:32,0,...,기타,0,94710,43066,False,False,True,1,400,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46787,46744,카드,36300.0,교환안함,0,D-20171229-0000036-00,무료,2017-12-29 오후 4:32,2018-01-02 오전 5:31,0,...,국민,0,52800,3170,False,False,True,1,200,용기
46788,46745,카드,36300.0,교환안함,0,D-20171229-0000043-00,무료,2018-01-03 오전 9:45,2018-01-07 오전 5:38,0,...,비씨,0,56100,3172,False,False,True,2,400,용기
46789,46746,카드,22280.0,교환안함,300,D-20171230-0000022-01,무료,2018-01-02 오후 7:06,2018-01-11 오전 2:46,0,...,기타,0,36000,3177,False,False,False,1,1500,기타
46790,46747,카드,55000.0,교환안함,320,D-20171231-0000017-01,무료,2018-01-03 오후 2:00,2018-01-06 오후 6:06,0,...,기타,0,73460,3178,False,False,False,1,400,용기


In [30]:
df_merge.isnull().sum()

Unnamed: 0        0
결제수단              0
공급원가              0
교환구분              0
네이버 포인트           0
배송번호              0
배송비 정보            0
배송시작일             0
배송완료일             0
사용한 적립금액          0
상품구매금액            0
상품명               0
상품번호              0
상품별 추가할인금액        0
수령인 주소            8
주문경로              0
주문번호              0
주문일시              0
주문자ID          3105
카드사               0
쿠폰 할인금액           0
판매가               0
품목번호              0
친환경               0
사은품               0
회원                0
주문수량              0
총수량               0
분류카테고리        17321
dtype: int64

In [31]:
df_nan = df_merge[df_merge['분류카테고리'].isnull()]
# df_nan = df_merge['분류카테고리'].isnull()
df_nan

,Unnamed: 0,결제수단,공급원가,교환구분,네이버 포인트,배송번호,배송비 정보,배송시작일,배송완료일,사용한 적립금액,...,카드사,쿠폰 할인금액,판매가,품목번호,친환경,사은품,회원,주문수량,총수량,분류카테고리
4,4,무통장입금,61600.0,교환안함,0,D-20181231-0000108-00,무료,2019-01-07 오전 8:56,2019-01-11 오전 5:32,0,...,기타,0,94710,43066,False,False,True,1,400,NaN
9,9,무통장입금,79200.0,교환안함,0,D-20181231-0000184-03,무료,2019-01-03 오전 9:30,2019-01-07 오전 5:32,0,...,기타,0,121110,43101,False,False,True,1,400,NaN
10,10,무통장입금,61600.0,교환안함,0,D-20181231-0000205-00,무료,2019-01-03 오전 9:30,2019-01-07 오전 5:32,0,...,기타,0,94710,43110,False,False,True,1,200,NaN
11,11,무통장입금,56100.0,교환안함,0,D-20181231-0000205-01,무료,2019-01-03 오전 9:30,2019-01-07 오전 5:32,0,...,기타,0,107910,43111,False,False,True,1,200,NaN
15,15,무통장입금,79200.0,교환안함,0,D-20181231-0000205-05,무료,2019-01-03 오전 9:30,2019-01-07 오전 5:32,0,...,기타,0,121110,43115,False,False,True,1,400,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46768,46725,카드,98560.0,교환안함,0,D-20171228-0000067-00,무료,2017-12-28 오후 6:20,2018-01-01 오전 5:37,0,...,롯데,0,130950,3146,False,False,False,1,1600,NaN
46774,46731,휴대폰,15500.0,교환안함,0,D-20171228-0000092-01,무료,2017-12-28 오후 6:20,2018-01-01 오전 5:37,0,...,기타,0,28210,3152,False,False,True,1,100,NaN
46783,46740,카드,55440.0,교환안함,276,D-20171228-0000132-04,무료,2017-12-29 오후 4:34,2018-01-07 오전 4:55,0,...,기타,0,69960,3165,False,False,False,1,1200,NaN
46784,46741,카드,45760.0,교환안함,276,D-20171228-0000132-05,무료,2017-12-29 오후 4:34,2018-01-07 오전 4:55,0,...,기타,0,63140,3166,False,False,False,1,800,NaN


In [21]:
# df_merge.loc[df_merge['상품명']=='포기김치용기 투명 100개 1set(2박스/6.5L)']

In [ ]:
cnt = 0
for r in df_nan:
    print()
    cnt += 1
    if cnt == 10:
        break

In [ ]:
df_merge.iloc[1]['상품번호']

In [ ]:
df_product

In [ ]:
df_merge.iloc[189]

In [46]:
type(df_nan['상품번호'][4])

str

In [48]:
type(df_product['상품코드'][1])

str

In [32]:
import math

In [33]:
for v in df_nan.iloc:
#     print(idx)
    print(v['Unnamed: 0'])
    idx = v['Unnamed: 0']
    print(idx)
    print('################', df_nan.iloc[idx]['상품번호'])
    print(type(df_nan.iloc[idx]['상품번호']))
    for p in df_product.loc:
        print(p)
        if p['상품코드'] == df_nan.iloc[idx]['상품번호']:
            df_merge.iloc[idx]['분류카테고리'] = p['분류카테고리']
            break
#     print(type(v))
#     try:
#         if math.isnan(v):
#             for p in df_product.loc:
#                 print(p)
#                 if p['상품코드'] == df_merge.iloc[idx]['상품번호']:
#                     df_merge.iloc[idx]['분류카테고리'] = p['분류카테고리']
#                     break
                
#     except:
#         print('스트링')

#     if type(v) != 'str':
#         print(math.isnan(v))
#     if v.isnull():
#         print('nullll')
#     if v == 'nan': # 카테고리가 Nan일 경우
#         for p in df_product.loc:
#             print(p)
# #             print(idx)
# #             print('###', p['상품코드'])
    
# #             print('3333', df_merge.iloc[idx]['상품번호'])
#             if p['상품코드'] == df_merge.iloc[idx]['상품번호']:
#                 print('!!!!!!!!!!')
#                 df_merge.iloc[idx]['분류카테고리'] = p['분류카테고리']
#                 break
#     print(idx, nan)
#     print(type(nan))

4
4
################ 1037.0
<class 'str'>
Unnamed: 0                                                    0
#                                                             1
공급사                                                         패커스
과세/면세                                                        과세
담기                                                            0
매입가                                                       55440
상품명(기본)                            실링용기 71호 백색 2,200ml 400개 1박스
상품코드                                                     2184.0
재입고알림                                                       사용함
카테고리          018 : 실링용기\n018002 : 실링용기 > 2319(2318)시리즈\n042...
판매가                                                       77480
품절여부                                                         정상
후기                                                            0
분류카테고리                                                       용기
Name: 0, dtype: object
Unnamed: 0                             

KeyError: 189

In [ ]:
df_merge.isnull().sum()

In [ ]:
for a in df_product.loc:
#     print(a)
    print(a['상품코드'] == 2184)
    break

In [ ]:
df_product(df_product['상품코드'] == 42)

In [ ]:
for a in df_product.loc:
#             print(p)
#             print(idx)
#             print(type(p['상품코드']))
#             print('3333', type(df_merge.iloc[idx]['상품번호']))
    if a['상품코드'] == 42:
        print(a)
        break

In [ ]:
df_product.loc[df_product['상품코드'] == 1969]

In [23]:
df_merge.iloc[1]['분류카테고리']

'용기'

In [ ]:
for idx, nan in enumerate(df_merge['분류카테고리'].iloc):
    print(idx)
    print(nan)
    print(type(nan))

In [4]:
df_product = pd.read_csv("./DataSet/log_order_with_category.csv", engine='python')
df_product.shape

(33570, 30)